# Step 1: Make a dataset 

In [64]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import pickle

In [2]:
def get_list_of_strokes():
    list_of_strokes = ['Serve Forehand Backspin',
                   'Serve Forehand Loop',
                   'Serve Forehand Sidespin',
                   'Serve Forehand Topspin',

                   'Serve Backhand Backspin',
                   'Serve Backhand Loop',
                   'Serve Backhand Sidespin',
                   'Serve Backhand Topspin',

                   'Offensive Forehand Hit',
                   'Offensive Forehand Loop',
                   'Offensive Forehand Flip',

                   'Offensive Backhand Hit',
                   'Offensive Backhand Loop',
                   'Offensive Backhand Flip',

                   'Defensive Forehand Push',
                   'Defensive Forehand Block',
                   'Defensive Forehand Backspin',

                   'Defensive Backhand Push',
                   'Defensive Backhand Block',
                   'Defensive Backhand Backspin',
                   'Negative']
    return list_of_strokes


In [3]:
dataset_path = os.listdir("C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas")
dataset_path

['Defensive Backhand Backspin',
 'Defensive Backhand Block',
 'Defensive Backhand Push',
 'Defensive Forehand Backspin',
 'Defensive Forehand Block',
 'Defensive Forehand Push',
 'Negative',
 'Offensive Backhand Flip',
 'Offensive Backhand Hit',
 'Offensive Backhand Loop',
 'Offensive Forehand Flip',
 'Offensive Forehand Hit',
 'Offensive Forehand Loop',
 'Serve Backhand Backspin',
 'Serve Backhand Loop',
 'Serve Backhand Sidespin',
 'Serve Backhand Topspin',
 'Serve Forehand Backspin',
 'Serve Forehand Loop',
 'Serve Forehand Sidespin',
 'Serve Forehand Topspin']

In [ ]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
train_df.shape

In [5]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Defensive Backhand Backspin', 'Defensive Backhand Block', 'Defensive Backhand Push', 'Defensive Forehand Backspin', 'Defensive Forehand Block', 'Defensive Forehand Push', 'Negative', 'Offensive Backhand Flip', 'Offensive Backhand Hit', 'Offensive Backhand Loop', 'Offensive Forehand Flip', 'Offensive Forehand Hit', 'Offensive Forehand Loop', 'Serve Backhand Backspin', 'Serve Backhand Loop', 'Serve Backhand Sidespin', 'Serve Backhand Topspin', 'Serve Forehand Backspin', 'Serve Forehand Loop', 'Serve Forehand Sidespin', 'Serve Forehand Topspin']


array([[ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],

# Step 2: Extract and normalize frames from videos

In [6]:
def extract_frames(video_path, num_frames=16, resize_dim=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=np.int32)
    frames = []
    for index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, resize_dim)
            frames.append(frame)
    cap.release()
    frames = preprocess_frames(frames)
    return np.array(frames)

def preprocess_frames(frames):
    frames = frames / 255.0
    return frames

In [52]:


# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def extract_features(video_path, num_frames=16, resize_dim=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=np.int32)
    features = []
    for index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, resize_dim)
            frame = preprocess_frames(frame)
            x = image.img_to_array(frame)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            frame_features = model.predict(x)
            features.append(frame_features)
    cap.release()
    features = np.array(features)
    features = features.reshape((features.shape[0], -1))
    return features


In [65]:


def define_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def train_model(model, X_train, y_train, epochs=30, batch_size=32):
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train,
                        validation_split= 0.1,
                        epochs=epochs,
                        batch_size=batch_size)
    return history

In [26]:
rooms = []
for item in dataset_path:
 # Get all the file names
	all_rooms = os.listdir('C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas' + '/' +item+'/')
	rooms.append(all_rooms)

In [19]:
 # Add them to the list
rooms = []
for room in all_rooms:
    rooms.append((item, str(item) + '/' + room))

In [40]:
for vid in rooms:
    for clip in vid:
        print(clip)

10410755888.mp4
10644410824.mp4
115268294.mp4
1431034833.mp4
2289610766.mp4
3101413152.mp4
351154895.mp4
357494546.mp4
4102951825.mp4
45710692103.mp4
47310101010310.mp4
510377410510.mp4
5545916710.mp4
5857686108.mp4
6919410891.mp4
7641017885.mp4
795925254.mp4
82510610489.mp4
8348883210.mp4
8858181067.mp4
947687719.mp4
9578102826.mp4
index.html
index.html@C=D;O=A
index.html@C=M;O=A
index.html@C=N;O=D
index.html@C=S;O=A
10826710557.mp4
16347910106.mp4
2103238533.mp4
2286672410.mp4
2924106843.mp4
314984131.mp4
3319284107.mp4
35781041053.mp4
523356858.mp4
626569467.mp4
648126876.mp4
65105107757.mp4
656624451.mp4
737362535.mp4
799437114.mp4
81084631067.mp4
8781044352.mp4
911231462.mp4
9514102672.mp4
index.html
index.html@C=D;O=A
index.html@C=M;O=A
index.html@C=N;O=D
index.html@C=S;O=A
2444649104.mp4
310106982101.mp4
4286910212.mp4
4715458110.mp4
675258964.mp4
782355349.mp4
index.html
index.html@C=D;O=A
index.html@C=M;O=A
index.html@C=N;O=D
index.html@C=S;O=A
1010108272102.mp4
1013589711.mp4

In [44]:
rooms[0]

['10410755888.mp4',
 '10644410824.mp4',
 '115268294.mp4',
 '1431034833.mp4',
 '2289610766.mp4',
 '3101413152.mp4',
 '351154895.mp4',
 '357494546.mp4',
 '4102951825.mp4',
 '45710692103.mp4',
 '47310101010310.mp4',
 '510377410510.mp4',
 '5545916710.mp4',
 '5857686108.mp4',
 '6919410891.mp4',
 '7641017885.mp4',
 '795925254.mp4',
 '82510610489.mp4',
 '8348883210.mp4',
 '8858181067.mp4',
 '947687719.mp4',
 '9578102826.mp4',
 'index.html',
 'index.html@C=D;O=A',
 'index.html@C=M;O=A',
 'index.html@C=N;O=D',
 'index.html@C=S;O=A']

In [54]:
#os.listdir('C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/')
# Define directory containing videos
video_dir = 'C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/'
num_frames = 16
resize_dim = (224, 224)

# Extract features from all videos in directory
features = []
labels = []
for video_path in os.listdir(video_dir ):
    print(video_path)
    print(os.path.join(video_dir, video_path))
    # Extract features from video
    video_features = extract_features(os.path.join(video_dir, video_path), num_frames=num_frames, resize_dim=resize_dim)
    features.append(video_features)
    # Extract label from filename (assuming filename format: "class_label_video_id.mp4")
    label = video_path.split("_")[0]
    labels.append(label)
features = np.concatenate(features)
labels = np.array(labels)


10410755888.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/10410755888.mp4
1/1 [==============================] - 0s 151ms/step
10644410824.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/10644410824.mp4
1/1 [==============================] - 0s 145ms/step
115268294.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/115268294.mp4
1/1 [==============================] - 0s 150ms/step
1431034833.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/1431034833.mp4
1/1 [==============================] - 0s 147ms/step
2289610766.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/2289610766.mp4
1/1 [==============================] - 0s 148ms/step
3101413152.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/3101413152.mp4
1/1 [==============================] - 0s 144ms/step
351154895.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/c

1/1 [==============================] - 0s 148ms/step
45710692103.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/45710692103.mp4
1/1 [==============================] - 0s 148ms/step
47310101010310.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/47310101010310.mp4
1/1 [==============================] - 0s 130ms/step
510377410510.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/510377410510.mp4
1/1 [==============================] - 0s 140ms/step
5545916710.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/5545916710.mp4
1/1 [==============================] - 0s 147ms/step
5857686108.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/5857686108.mp4
1/1 [==============================] - 0s 142ms/step
6919410891.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/6919410891.mp4
1/1 [==============================] - 0s 14

1/1 [==============================] - 0s 151ms/step
8348883210.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/8348883210.mp4
1/1 [==============================] - 0s 146ms/step
8858181067.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/8858181067.mp4
1/1 [==============================] - 0s 144ms/step
947687719.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/947687719.mp4
1/1 [==============================] - 0s 150ms/step
9578102826.mp4
C:/Users/Uzair Siddiqui/jupyter/FYP/data/clas/Defensive Backhand Backspin/9578102826.mp4
1/1 [==============================] - 0s 141ms/step


(352, 100352)

In [ ]:
# Train model on extracted features
num_classes = len(np.unique(labels))
input_shape = features.shape[1:]
model = define_model(input_shape=input_shape, num_classes=num_classes)
train_model(model, features, labels, epochs=30, batch_size=32)